In [6]:
from lenskit.metrics import dataGenerator
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn
from lenskit import topn, topnFair

ModuleNotFoundError: No module named 'dataGenerator'

In [5]:
import numpy as np
import pandas as pd
%matplotlib inline

In [3]:
#ratings = pd.read_csv('/Users/denisehansen/Desktop/ITU/Thesis/Fair-Recommendations/ml-latest-small/ratings.1.csv', sep=',',
#                      names=['user', 'item', 'rating', 'timestamp'], header=0)

#items = pd.read_csv('/Users/denisehansen/Desktop/ITU/Thesis/Fair-Recommendations/ml-latest-small/movies.csv', sep=',',
#                      names=['movieId','title','genres'], header=0)

ratings = pd.read_csv('/Users/josse/Git-kode projects/Speciale/Fair-Recommendations/ml-latest-small/ratings.1.csv', sep=',',
                      names=['user', 'item', 'rating', 'timestamp'], header=0)

items = pd.read_csv('/Users/josse/Git-kode projects/Speciale/Fair-Recommendations/ml-latest-small/movies.csv', sep=',',
                      names=['item','title','genres'], header=0)

#C:\Users\josse\Git-kode projects\Speciale\Fair-Recommendations\ml-latest-small\ratings.csv

In [4]:
#items

In [5]:
items_dummy = pd.concat([items, items['genres'].str.get_dummies(sep='|')], axis=1)
items_dummy.head()

,item,title,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
algo_ii = knn.ItemItem(20)
algo_als = als.BiasedMF(50)

In [7]:
def eval(aname, algo, train, test):
    fittable = util.clone(algo)
    fittable = Recommender.adapt(fittable)
    fittable.fit(train)
    users = test.user.unique()
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [8]:
all_recs = []
test_data = []
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 1, xf.SampleFrac(0.2)):
    test_data.append(test)
    all_recs.append(eval('ItemItem', algo_ii, train, test))
    all_recs.append(eval('ALS', algo_als, train, test))
    

test _reccomend
test _reccomend


In [9]:
all_recs = pd.concat(all_recs, ignore_index=True)
all_recs_joined = all_recs.join(items_dummy.set_index('item'), on='item')
all_recs_joined.head()

,item,score,user,rank,Algorithm,title,genres,(no genres listed),Action,Adventure,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1968,3.5,1,1,ItemItem,"Breakfast Club, The (1985)",Comedy|Drama,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1961,3.5,1,2,ItemItem,Rain Man (1988),Drama,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,480,3.5,1,3,ItemItem,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,0,1,1,...,0,0,0,0,0,0,1,1,0,0
3,349,3.5,1,4,ItemItem,Clear and Present Danger (1994),Action|Crime|Drama|Thriller,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,296,3.5,1,5,ItemItem,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [10]:
test_data = pd.concat(test_data, ignore_index=True)
test_data.head()

,user,item,rating
0,1,1371,2.5
1,1,2294,2.0
2,1,2455,2.5
3,1,1293,2.0
4,2,720,4.0


In [11]:
#all_recs_joined['item'].values

In [12]:
_protected_group2 = all_recs_joined.loc[all_recs_joined['Action'] == 1]
_protected_group2.head()
_protected_group2['item'].values

array([  480,   349,  2716,  8636,  1953,  2716,  1544,  2193,   349,
         480,  1953,  1544,  2193,   592,  8636,   377,   480,   349,
         592,  1953,  2716,   377,  2193,   589,   592,  1197,   349,
         480,  1220,  2406,  2194,  2115,  1036,  1396,  1240,  1210,
        1387,  1208,  1200,  1198,  1196,  1858,  2000,  3265,  2991,
         541,   112,  1197,  1953,  1240,  1220,  2406,  1210,  2640,
        2628,  2194,  1208,  1200,  2723,  1198,  1196,  1387,  1396,
        1858,  2000,  2115,  2991,   541,  1036,  3265,   112,  2959,
         589,  1396,  2115,  2194,  2406,  2628,  2000,   541,  1210,
        1220,  1240,  1200,  1198,  1196,  1387,  1036,  1858,  2640,
        1208,  2991,  2723,  3265,   112,   349,   480,  1953,  2959,
        2028,  2273,   367, 41569,  8644,  3753,   110,   736,  1580,
         485,   589,   592,  1953,  2194,  2000,  2115,  1036,  1240,
        1220,  1387,  1396,  1210,  1208,  1858,  1200,  1198,  1196,
        2991,  2723,

In [2]:
# user , algorithms = iput to group_cols --> de koloner der skal  groupes på 
rla = topnFair.FairRecListAnalysis(['user', 'Algorithm'])
rla.add_metric(rND)
results = rla.compute(all_recs_joined, test_data)
results.head()

NameError: name 'topnFair' is not defined

In [ ]:
results.groupby('Algorithm').ndcg.mean()

In [ ]:
results.groupby('Algorithm').ndcg.mean().plot.bar()